In [1]:
import time
import os
import json
import pickle
import tensorflow as tf
from tensorflow.keras import utils
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import ResNet50, VGG19
from tensorflow.keras.applications.imagenet_utils import preprocess_input


from matplotlib.pyplot import imread, imshow

import h5py
import numpy as np
import matplotlib.pyplot as plt
import random
from datetime import datetime
import nibabel as nib
import re
from collections import Counter


%matplotlib inline
%load_ext autoreload
%autoreload 1

SEED=1
random.seed(SEED)
np.random.seed(SEED)
#tf.set_random_seed(SEED)
tf.random.set_seed

K.clear_session()



In [19]:

# Get fmri ROI data
import h5py
roi_list = []
ax_length = 200
for roi in ['RHPPA']:
    tr_list = []
    for TR in ['TR3']:
        with h5py.File('/home/ubuntu/boldroi/ROIs/CSI1/h5/CSI1_ROIs_%s.h5' % TR, 'r') as f:
            print(list(f.keys()))
            x_roi = list(f[roi])
            x_r = np.stack(x_roi, axis=0)
            tr_list.append(x_r)
            #mean_x = np.mean(x_r, axis=0)
            #std_x = np.std(x_r, axis=0, ddof=1)
            #xt = (x_r - mean_x) / std_x
            #tr_list.append(xt)
            #xt_pad = np.pad(xt, ((0, 0), (0, ax_length-xt.shape[1])), mode='constant', constant_values=0)
            #tr_list.append(xt_pad)

    x_stack = np.dstack(tr_list)
    xsa = np.swapaxes(x_stack,1,2)
    roi_list.append(xsa)

x_all = np.concatenate(roi_list)
x_all =np.squeeze(x_all)
print(x_all.shape)


['LHEarlyVis', 'LHLOC', 'LHOPA', 'LHPPA', 'LHRSC', 'RHEarlyVis', 'RHLOC', 'RHOPA', 'RHPPA', 'RHRSC']
(5254, 200)


In [44]:
# load stimuli list
with open('/home/ubuntu/boldroi/ROIs/stim_lists/CSI01_stim_lists.txt', 'r') as f:
    stimuli_list = f.read().splitlines()

classes = {'ImageNet': 0, 'COCO': 1, 'Scene': 2}
stimulusDirPath = '/home/ubuntu/fmriNet/fmriNet/images/BOLD5000_Stimuli/Scene_Stimuli/Presented_Stimuli'
stimuli_file_list = []
label_list = []
for imageFileName in stimuli_list:
    for (currDir, _, fileList) in os.walk(stimulusDirPath):
        currBaseDir = os.path.basename(currDir)
        for filename in fileList:
            if filename in imageFileName:
                stimuli_file_list.append(os.path.join(currDir, filename))
                label_list.append(classes.get(currBaseDir))
                break

labels = np.asarray(label_list)
print(labels.shape)
labels_all = utils.to_categorical(labels)

(5254,)


In [8]:
# preprocess stimuli images
pstimuli = []
for imgFile in stimuli_file_list:
    img = image.load_img(imgFile, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    pstimuli.append(preprocess_input(x))

In [9]:
# Get image features from resnet
base_model = ResNet50(weights='imagenet')
#base_model.summary()
b_model = Model(inputs = base_model.input, outputs = base_model.get_layer('avg_pool').output)
b_model.summary()



Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [10]:
# Use pretrained model to get stimuli activations

image_activations = []
num_samples = len(pstimuli)
# precompute x_images with above pretrainded model
for index, x in enumerate(pstimuli):
    print("Computing image: %s out of %s" % (index, num_samples))
    image_activations.append(b_model.predict(x))

#print(len(image_activations))
#print(image_activations[0].shape)


Computing image: 0 out of 5254
Computing image: 1 out of 5254
Computing image: 2 out of 5254
Computing image: 3 out of 5254
Computing image: 4 out of 5254
Computing image: 5 out of 5254
Computing image: 6 out of 5254
Computing image: 7 out of 5254
Computing image: 8 out of 5254
Computing image: 9 out of 5254
Computing image: 10 out of 5254
Computing image: 11 out of 5254
Computing image: 12 out of 5254
Computing image: 13 out of 5254
Computing image: 14 out of 5254
Computing image: 15 out of 5254
Computing image: 16 out of 5254
Computing image: 17 out of 5254
Computing image: 18 out of 5254
Computing image: 19 out of 5254
Computing image: 20 out of 5254
Computing image: 21 out of 5254
Computing image: 22 out of 5254
Computing image: 23 out of 5254
Computing image: 24 out of 5254
Computing image: 25 out of 5254
Computing image: 26 out of 5254
Computing image: 27 out of 5254
Computing image: 28 out of 5254
Computing image: 29 out of 5254
Computing image: 30 out of 5254
Computing image: 3

Computing image: 253 out of 5254
Computing image: 254 out of 5254
Computing image: 255 out of 5254
Computing image: 256 out of 5254
Computing image: 257 out of 5254
Computing image: 258 out of 5254
Computing image: 259 out of 5254
Computing image: 260 out of 5254
Computing image: 261 out of 5254
Computing image: 262 out of 5254
Computing image: 263 out of 5254
Computing image: 264 out of 5254
Computing image: 265 out of 5254
Computing image: 266 out of 5254
Computing image: 267 out of 5254
Computing image: 268 out of 5254
Computing image: 269 out of 5254
Computing image: 270 out of 5254
Computing image: 271 out of 5254
Computing image: 272 out of 5254
Computing image: 273 out of 5254
Computing image: 274 out of 5254
Computing image: 275 out of 5254
Computing image: 276 out of 5254
Computing image: 277 out of 5254
Computing image: 278 out of 5254
Computing image: 279 out of 5254
Computing image: 280 out of 5254
Computing image: 281 out of 5254
Computing image: 282 out of 5254
Computing 

Computing image: 503 out of 5254
Computing image: 504 out of 5254
Computing image: 505 out of 5254
Computing image: 506 out of 5254
Computing image: 507 out of 5254
Computing image: 508 out of 5254
Computing image: 509 out of 5254
Computing image: 510 out of 5254
Computing image: 511 out of 5254
Computing image: 512 out of 5254
Computing image: 513 out of 5254
Computing image: 514 out of 5254
Computing image: 515 out of 5254
Computing image: 516 out of 5254
Computing image: 517 out of 5254
Computing image: 518 out of 5254
Computing image: 519 out of 5254
Computing image: 520 out of 5254
Computing image: 521 out of 5254
Computing image: 522 out of 5254
Computing image: 523 out of 5254
Computing image: 524 out of 5254
Computing image: 525 out of 5254
Computing image: 526 out of 5254
Computing image: 527 out of 5254
Computing image: 528 out of 5254
Computing image: 529 out of 5254
Computing image: 530 out of 5254
Computing image: 531 out of 5254
Computing image: 532 out of 5254
Computing 

Computing image: 753 out of 5254
Computing image: 754 out of 5254
Computing image: 755 out of 5254
Computing image: 756 out of 5254
Computing image: 757 out of 5254
Computing image: 758 out of 5254
Computing image: 759 out of 5254
Computing image: 760 out of 5254
Computing image: 761 out of 5254
Computing image: 762 out of 5254
Computing image: 763 out of 5254
Computing image: 764 out of 5254
Computing image: 765 out of 5254
Computing image: 766 out of 5254
Computing image: 767 out of 5254
Computing image: 768 out of 5254
Computing image: 769 out of 5254
Computing image: 770 out of 5254
Computing image: 771 out of 5254
Computing image: 772 out of 5254
Computing image: 773 out of 5254
Computing image: 774 out of 5254
Computing image: 775 out of 5254
Computing image: 776 out of 5254
Computing image: 777 out of 5254
Computing image: 778 out of 5254
Computing image: 779 out of 5254
Computing image: 780 out of 5254
Computing image: 781 out of 5254
Computing image: 782 out of 5254
Computing 

Computing image: 1003 out of 5254
Computing image: 1004 out of 5254
Computing image: 1005 out of 5254
Computing image: 1006 out of 5254
Computing image: 1007 out of 5254
Computing image: 1008 out of 5254
Computing image: 1009 out of 5254
Computing image: 1010 out of 5254
Computing image: 1011 out of 5254
Computing image: 1012 out of 5254
Computing image: 1013 out of 5254
Computing image: 1014 out of 5254
Computing image: 1015 out of 5254
Computing image: 1016 out of 5254
Computing image: 1017 out of 5254
Computing image: 1018 out of 5254
Computing image: 1019 out of 5254
Computing image: 1020 out of 5254
Computing image: 1021 out of 5254
Computing image: 1022 out of 5254
Computing image: 1023 out of 5254
Computing image: 1024 out of 5254
Computing image: 1025 out of 5254
Computing image: 1026 out of 5254
Computing image: 1027 out of 5254
Computing image: 1028 out of 5254
Computing image: 1029 out of 5254
Computing image: 1030 out of 5254
Computing image: 1031 out of 5254
Computing imag

...

Computing image: 4148 out of 5254
Computing image: 4149 out of 5254
Computing image: 4150 out of 5254
Computing image: 4151 out of 5254
Computing image: 4152 out of 5254
Computing image: 4153 out of 5254
Computing image: 4154 out of 5254
Computing image: 4155 out of 5254
Computing image: 4156 out of 5254
Computing image: 4157 out of 5254
Computing image: 4158 out of 5254
Computing image: 4159 out of 5254
Computing image: 4160 out of 5254
Computing image: 4161 out of 5254
Computing image: 4162 out of 5254
Computing image: 4163 out of 5254
Computing image: 4164 out of 5254
Computing image: 4165 out of 5254
Computing image: 4166 out of 5254
Computing image: 4167 out of 5254
Computing image: 4168 out of 5254
Computing image: 4169 out of 5254
Computing image: 4170 out of 5254
Computing image: 4171 out of 5254
Computing image: 4172 out of 5254
Computing image: 4173 out of 5254
Computing image: 4174 out of 5254
Computing image: 4175 out of 5254
Computing image: 4176 out of 5254
Computing imag

Computing image: 4390 out of 5254
Computing image: 4391 out of 5254
Computing image: 4392 out of 5254
Computing image: 4393 out of 5254
Computing image: 4394 out of 5254
Computing image: 4395 out of 5254
Computing image: 4396 out of 5254
Computing image: 4397 out of 5254
Computing image: 4398 out of 5254
Computing image: 4399 out of 5254
Computing image: 4400 out of 5254
Computing image: 4401 out of 5254
Computing image: 4402 out of 5254
Computing image: 4403 out of 5254
Computing image: 4404 out of 5254
Computing image: 4405 out of 5254
Computing image: 4406 out of 5254
Computing image: 4407 out of 5254
Computing image: 4408 out of 5254
Computing image: 4409 out of 5254
Computing image: 4410 out of 5254
Computing image: 4411 out of 5254
Computing image: 4412 out of 5254
Computing image: 4413 out of 5254
Computing image: 4414 out of 5254
Computing image: 4415 out of 5254
Computing image: 4416 out of 5254
Computing image: 4417 out of 5254
Computing image: 4418 out of 5254
Computing imag

Computing image: 4632 out of 5254
Computing image: 4633 out of 5254
Computing image: 4634 out of 5254
Computing image: 4635 out of 5254
Computing image: 4636 out of 5254
Computing image: 4637 out of 5254
Computing image: 4638 out of 5254
Computing image: 4639 out of 5254
Computing image: 4640 out of 5254
Computing image: 4641 out of 5254
Computing image: 4642 out of 5254
Computing image: 4643 out of 5254
Computing image: 4644 out of 5254
Computing image: 4645 out of 5254
Computing image: 4646 out of 5254
Computing image: 4647 out of 5254
Computing image: 4648 out of 5254
Computing image: 4649 out of 5254
Computing image: 4650 out of 5254
Computing image: 4651 out of 5254
Computing image: 4652 out of 5254
Computing image: 4653 out of 5254
Computing image: 4654 out of 5254
Computing image: 4655 out of 5254
Computing image: 4656 out of 5254
Computing image: 4657 out of 5254
Computing image: 4658 out of 5254
Computing image: 4659 out of 5254
Computing image: 4660 out of 5254
Computing imag

Computing image: 4874 out of 5254
Computing image: 4875 out of 5254
Computing image: 4876 out of 5254
Computing image: 4877 out of 5254
Computing image: 4878 out of 5254
Computing image: 4879 out of 5254
Computing image: 4880 out of 5254
Computing image: 4881 out of 5254
Computing image: 4882 out of 5254
Computing image: 4883 out of 5254
Computing image: 4884 out of 5254
Computing image: 4885 out of 5254
Computing image: 4886 out of 5254
Computing image: 4887 out of 5254
Computing image: 4888 out of 5254
Computing image: 4889 out of 5254
Computing image: 4890 out of 5254
Computing image: 4891 out of 5254
Computing image: 4892 out of 5254
Computing image: 4893 out of 5254
Computing image: 4894 out of 5254
Computing image: 4895 out of 5254
Computing image: 4896 out of 5254
Computing image: 4897 out of 5254
Computing image: 4898 out of 5254
Computing image: 4899 out of 5254
Computing image: 4900 out of 5254
Computing image: 4901 out of 5254
Computing image: 4902 out of 5254
Computing imag

Computing image: 5116 out of 5254
Computing image: 5117 out of 5254
Computing image: 5118 out of 5254
Computing image: 5119 out of 5254
Computing image: 5120 out of 5254
Computing image: 5121 out of 5254
Computing image: 5122 out of 5254
Computing image: 5123 out of 5254
Computing image: 5124 out of 5254
Computing image: 5125 out of 5254
Computing image: 5126 out of 5254
Computing image: 5127 out of 5254
Computing image: 5128 out of 5254
Computing image: 5129 out of 5254
Computing image: 5130 out of 5254
Computing image: 5131 out of 5254
Computing image: 5132 out of 5254
Computing image: 5133 out of 5254
Computing image: 5134 out of 5254
Computing image: 5135 out of 5254
Computing image: 5136 out of 5254
Computing image: 5137 out of 5254
Computing image: 5138 out of 5254
Computing image: 5139 out of 5254
Computing image: 5140 out of 5254
Computing image: 5141 out of 5254
Computing image: 5142 out of 5254
Computing image: 5143 out of 5254
Computing image: 5144 out of 5254
Computing imag

In [15]:

activations_all = np.concatenate(image_activations)
print(activations_all.shape)

!mkdir -p 'imageActivation'
with open('imageActivation/renet_avg_pool_activations.npy', 'wb') as f:
    pickle.dump(activations_all, f)

del activations_all

(5254, 2048)


In [17]:

# load stimuli activations
with open('imageActivation/renet_avg_pool_activations.npy', 'rb') as f:
    y_all = pickle.load(f)

In [21]:
# Split data into train/test set
#from sklearn.utils import shuffle
num_samples = x_all.shape[0]
num_voxels = 100  # maximum number of voxels to use for training
div = int(num_samples * 0.9)
print("Division index: %s" % div)
#x_shuffle, y_shuffle = shuffle(x_all, y_all, random_state=0)
x_train = x_all[0:div, :]
y_train = y_all[0:div, :]
x_test = x_all[div:, :]
y_test = y_all[div:, :]
print("x train shape: %s" % str(x_train.shape))
print("y train shape: %s" % str(y_train.shape))
print("x test shape: %s" % str(x_test.shape))
print("y test shape: %s" % str(y_test.shape))



Division index: 4728
x train shape: (4728, 200)
y train shape: (4728, 2048)
x test shape: (526, 200)
y test shape: (526, 2048)


In [27]:

# borrowed from bdpy https://github.com/KamitaniLab/bdpy 
from numpy.matlib import repmat
def corrcoef(x, y, var='row'):
    """
    Returns correlation coefficients between `x` and `y`

    Parameters
    ----------
    x, y : array_like
        Matrix or vector
    var : str, 'row' or 'col'
        Specifying whether rows (default) or columns represent variables

    Returns
    -------
    r
        Correlation coefficients
    """

    # Convert vectors to arrays
    if x.ndim == 1:
        x = np.array([x])

    if y.ndim == 1:
        y = np.array([y])

    # Normalize x and y to row-var format
    if var == 'row':
        # 'rowvar=1' in np.corrcoef

        # Vertical vector --> horizontal
        if x.shape[1] == 1:
            x = x.T
        if y.shape[1] == 1:
            y = y.T
    elif var == 'col':
        # 'rowvar=0' in np.corrcoef

        # Horizontal vector --> vertical
        if x.shape[0] == 1:
            x = x.T
        if y.shape[0] == 1:
            y = y.T

        # Convert to rowvar=1
        x = x.T
        y = y.T
    else:
        raise ValueError('Unknown var parameter specified')

    # Match size of x and y
    if x.shape[0] == 1 and y.shape[0] != 1:
        x = repmat(x, y.shape[0], 1)
        
    elif x.shape[0] != 1 and y.shape[0] == 1:
        y = repmat(y, x.shape[0], 1)

    # Check size of normalized x and y
    if x.shape != y.shape:
        raise TypeError('Input matrixes size mismatch')

    # Get num variables
    nvar = x.shape[0]

    # Get correlation
    rmat = np.corrcoef(x, y, rowvar=1)
    r = np.diag(rmat[:nvar, nvar:])

    return r

# borrowed from bdpy https://github.com/KamitaniLab/bdpy 
def select_top(data, value, num, axis=0, verbose=True):
    """
    Select top `num` features of `value` from `data`

    Parameters
    ----------
    data : array
       Data matrix
    value : array_like
       Vector of values
    num : int
       Number of selected features

    Returns
    -------
    selected_data : array
        Selected data matrix
    selected_index : array
        Index of selected data
    """

    if verbose:
        print_start_msg()

    num_elem = data.shape[axis]

    sorted_index = np.argsort(value)[::-1]

    rank = np.zeros(num_elem, dtype=np.int)
    rank[sorted_index] = np.array(range(0, num_elem))

    selected_index_bool = rank < num

    if axis == 0:
        selected_data = data[selected_index_bool, :]
        selected_index = np.array(range(0, num_elem), dtype=np.int)[selected_index_bool]
    elif axis == 1:
        selected_data = data[:, selected_index_bool]
        selected_index = np.array(range(0, num_elem), dtype=np.int)[selected_index_bool]
    else:
        raise ValueError('Invalid axis')

    if verbose:
        print_finish_msg()

    return selected_data, selected_index

# borrowed from bdpy https://github.com/KamitaniLab/bdpy 
def add_bias(x, axis=0):
    """
    Add bias terms to x

    Parameters
    ----------
    x : array_like
        Data matrix
    axis : 0 or 1, optional
        Axis in which bias terms are added (default: 0)

    Returns
    -------
    y : array_like
        Data matrix with bias terms
    """

    if axis == 0:
        vlen = x.shape[1]
        y = np.concatenate((x, np.array([np.ones(vlen)])), axis=0)
    elif axis == 1:
        vlen = x.shape[0]
        y = np.concatenate((x, np.array([np.ones(vlen)]).T), axis=1)
    else:
        raise ValueError('axis should be either 0 or 1')
    
    return y

In [28]:
from sklearn.linear_model import LinearRegression

# borrowed from https://github.com/KamitaniLab/GenericObjectDecoding
numFeatures = y_train.shape[1]
# Normalize brian data (x)
norm_mean_x = np.mean(x_train, axis=0)
norm_scale_x = np.std(x_train, axis=0, ddof=1)
x_train = (x_train - norm_mean_x) / norm_scale_x
x_test = (x_test - norm_mean_x) / norm_scale_x

y_true_list = []
y_pred_list = []
for i in range(numFeatures):

    print('Unit %03d' % (i + 1))
    start_time = time.time()

    # Get unit features
    y_train_unit = y_train[:, i]
    y_test_unit =  y_test[:, i]

    # Normalize image features for training (y_train_unit)
    norm_mean_y = np.mean(y_train_unit, axis=0)
    std_y = np.std(y_train_unit, axis=0, ddof=1)
    norm_scale_y = 1 if std_y == 0 else std_y

    y_train_unit = (y_train_unit - norm_mean_y) / norm_scale_y

    # Voxel selection
    corr = corrcoef(y_train_unit, x_train, var='col')

    x_train_unit, voxel_index = select_top(x_train, np.abs(corr), num_voxels, axis=1, verbose=False)
    x_test_unit = x_test[:, voxel_index]

    # Add bias terms
    x_train_unit = add_bias(x_train_unit, axis=1)
    x_test_unit = add_bias(x_test_unit, axis=1)

    # Setup regression
    # For quick demo, use linaer regression
    model = LinearRegression()
    #model = SparseLinearRegression(n_iter=n_iter, prune_mode=1)

    # Training and test
    try:
        model.fit(x_train_unit, y_train_unit)  # Training
        y_pred = model.predict(x_test_unit)    # Test
    except:
        # When SLiR failed, returns zero-filled array as predicted features
        y_pred = np.zeros(y_test_unit.shape)

    # Denormalize predicted features
    y_pred = y_pred * norm_scale_y + norm_mean_y

    y_true_list.append(y_test_unit)
    y_pred_list.append(y_pred)

    print('Time: %.3f sec' % (time.time() - start_time))

# Create numpy arrays for return values
y_predicted = np.vstack(y_pred_list).T
y_true = np.vstack(y_true_list).T

Unit 001
Time: 0.155 sec
Unit 002
Time: 0.097 sec
Unit 003
Time: 0.064 sec
Unit 004
Time: 0.062 sec
Unit 005
Time: 0.065 sec
Unit 006
Time: 0.066 sec
Unit 007
Time: 0.056 sec
Unit 008
Time: 0.062 sec
Unit 009
Time: 0.061 sec
Unit 010
Time: 0.066 sec
Unit 011
Time: 0.073 sec
Unit 012
Time: 0.063 sec
Unit 013
Time: 0.069 sec
Unit 014
Time: 0.066 sec
Unit 015
Time: 0.065 sec
Unit 016
Time: 0.174 sec
Unit 017
Time: 0.095 sec
Unit 018
Time: 0.099 sec
Unit 019
Time: 0.190 sec
Unit 020
Time: 0.094 sec
Unit 021
Time: 0.096 sec
Unit 022
Time: 0.064 sec
Unit 023
Time: 0.063 sec
Unit 024
Time: 0.064 sec
Unit 025
Time: 0.181 sec
Unit 026
Time: 0.096 sec
Unit 027
Time: 0.073 sec
Unit 028
Time: 0.065 sec
Unit 029
Time: 0.064 sec
Unit 030
Time: 0.069 sec
Unit 031
Time: 0.062 sec
Unit 032
Time: 0.066 sec
Unit 033
Time: 0.062 sec
Unit 034
Time: 0.063 sec
Unit 035
Time: 0.062 sec
Unit 036
Time: 0.064 sec
Unit 037
Time: 0.165 sec
Unit 038
Time: 0.084 sec
Unit 039
Time: 0.060 sec
Unit 040
Time: 0.169 sec


Time: 0.068 sec
Unit 330
Time: 0.058 sec
Unit 331
Time: 0.060 sec
Unit 332
Time: 0.067 sec
Unit 333
Time: 0.188 sec
Unit 334
Time: 0.108 sec
Unit 335
Time: 0.106 sec
Unit 336
Time: 0.087 sec
Unit 337
Time: 0.062 sec
Unit 338
Time: 0.065 sec
Unit 339
Time: 0.061 sec
Unit 340
Time: 0.062 sec
Unit 341
Time: 0.061 sec
Unit 342
Time: 0.064 sec
Unit 343
Time: 0.066 sec
Unit 344
Time: 0.164 sec
Unit 345
Time: 0.064 sec
Unit 346
Time: 0.065 sec
Unit 347
Time: 0.063 sec
Unit 348
Time: 0.066 sec
Unit 349
Time: 0.063 sec
Unit 350
Time: 0.063 sec
Unit 351
Time: 0.063 sec
Unit 352
Time: 0.089 sec
Unit 353
Time: 0.093 sec
Unit 354
Time: 0.069 sec
Unit 355
Time: 0.072 sec
Unit 356
Time: 0.064 sec
Unit 357
Time: 0.064 sec
Unit 358
Time: 0.064 sec
Unit 359
Time: 0.063 sec
Unit 360
Time: 0.060 sec
Unit 361
Time: 0.061 sec
Unit 362
Time: 0.064 sec
Unit 363
Time: 0.163 sec
Unit 364
Time: 0.100 sec
Unit 365
Time: 0.110 sec
Unit 366
Time: 0.061 sec
Unit 367
Time: 0.062 sec
Unit 368
Time: 0.062 sec
Unit 369


Time: 0.066 sec
Unit 660
Time: 0.063 sec
Unit 661
Time: 0.086 sec
Unit 662
Time: 0.099 sec
Unit 663
Time: 0.067 sec
Unit 664
Time: 0.057 sec
Unit 665
Time: 0.057 sec
Unit 666
Time: 0.060 sec
Unit 667
Time: 0.049 sec
Unit 668
Time: 0.058 sec
Unit 669
Time: 0.059 sec
Unit 670
Time: 0.071 sec
Unit 671
Time: 0.085 sec
Unit 672
Time: 0.106 sec
Unit 673
Time: 0.095 sec
Unit 674
Time: 0.111 sec
Unit 675
Time: 0.111 sec
Unit 676
Time: 0.094 sec
Unit 677
Time: 0.075 sec
Unit 678
Time: 0.074 sec
Unit 679
Time: 0.112 sec
Unit 680
Time: 0.121 sec
Unit 681
Time: 0.108 sec
Unit 682
Time: 0.077 sec
Unit 683
Time: 0.115 sec
Unit 684
Time: 0.105 sec
Unit 685
Time: 0.117 sec
Unit 686
Time: 0.102 sec
Unit 687
Time: 0.123 sec
Unit 688
Time: 0.106 sec
Unit 689
Time: 0.102 sec
Unit 690
Time: 0.097 sec
Unit 691
Time: 0.106 sec
Unit 692
Time: 0.083 sec
Unit 693
Time: 0.081 sec
Unit 694
Time: 0.089 sec
Unit 695
Time: 0.116 sec
Unit 696
Time: 0.154 sec
Unit 697
Time: 0.124 sec
Unit 698
Time: 0.117 sec
Unit 699


Time: 0.061 sec
Unit 990
Time: 0.056 sec
Unit 991
Time: 0.060 sec
Unit 992
Time: 0.061 sec
Unit 993
Time: 0.082 sec
Unit 994
Time: 0.099 sec
Unit 995
Time: 0.092 sec
Unit 996
Time: 0.061 sec
Unit 997
Time: 0.061 sec
Unit 998
Time: 0.092 sec
Unit 999
Time: 0.098 sec
Unit 1000
Time: 0.097 sec
Unit 1001
Time: 0.098 sec
Unit 1002
Time: 0.099 sec
Unit 1003
Time: 0.110 sec
Unit 1004
Time: 0.106 sec
Unit 1005
Time: 0.063 sec
Unit 1006
Time: 0.063 sec
Unit 1007
Time: 0.065 sec
Unit 1008
Time: 0.065 sec
Unit 1009
Time: 0.062 sec
Unit 1010
Time: 0.061 sec
Unit 1011
Time: 0.059 sec
Unit 1012
Time: 0.063 sec
Unit 1013
Time: 0.059 sec
Unit 1014
Time: 0.055 sec
Unit 1015
Time: 0.062 sec
Unit 1016
Time: 0.067 sec
Unit 1017
Time: 0.064 sec
Unit 1018
Time: 0.058 sec
Unit 1019
Time: 0.058 sec
Unit 1020
Time: 0.061 sec
Unit 1021
Time: 0.082 sec
Unit 1022
Time: 0.079 sec
Unit 1023
Time: 0.072 sec
Unit 1024
Time: 0.069 sec
Unit 1025
Time: 0.088 sec
Unit 1026
Time: 0.185 sec
Unit 1027
Time: 0.087 sec
Unit 1

Time: 0.064 sec
Unit 1308
Time: 0.062 sec
Unit 1309
Time: 0.055 sec
Unit 1310
Time: 0.058 sec
Unit 1311
Time: 0.061 sec
Unit 1312
Time: 0.058 sec
Unit 1313
Time: 0.087 sec
Unit 1314
Time: 0.070 sec
Unit 1315
Time: 0.070 sec
Unit 1316
Time: 0.161 sec
Unit 1317
Time: 0.097 sec
Unit 1318
Time: 0.100 sec
Unit 1319
Time: 0.103 sec
Unit 1320
Time: 0.102 sec
Unit 1321
Time: 0.060 sec
Unit 1322
Time: 0.063 sec
Unit 1323
Time: 0.062 sec
Unit 1324
Time: 0.061 sec
Unit 1325
Time: 0.086 sec
Unit 1326
Time: 0.061 sec
Unit 1327
Time: 0.062 sec
Unit 1328
Time: 0.066 sec
Unit 1329
Time: 0.064 sec
Unit 1330
Time: 0.087 sec
Unit 1331
Time: 0.063 sec
Unit 1332
Time: 0.074 sec
Unit 1333
Time: 0.064 sec
Unit 1334
Time: 0.062 sec
Unit 1335
Time: 0.062 sec
Unit 1336
Time: 0.065 sec
Unit 1337
Time: 0.061 sec
Unit 1338
Time: 0.062 sec
Unit 1339
Time: 0.062 sec
Unit 1340
Time: 0.065 sec
Unit 1341
Time: 0.063 sec
Unit 1342
Time: 0.062 sec
Unit 1343
Time: 0.088 sec
Unit 1344
Time: 0.092 sec
Unit 1345
Time: 0.065 

Time: 0.101 sec
Unit 1625
Time: 0.087 sec
Unit 1626
Time: 0.061 sec
Unit 1627
Time: 0.064 sec
Unit 1628
Time: 0.061 sec
Unit 1629
Time: 0.062 sec
Unit 1630
Time: 0.063 sec
Unit 1631
Time: 0.064 sec
Unit 1632
Time: 0.064 sec
Unit 1633
Time: 0.059 sec
Unit 1634
Time: 0.059 sec
Unit 1635
Time: 0.064 sec
Unit 1636
Time: 0.062 sec
Unit 1637
Time: 0.089 sec
Unit 1638
Time: 0.101 sec
Unit 1639
Time: 0.058 sec
Unit 1640
Time: 0.057 sec
Unit 1641
Time: 0.058 sec
Unit 1642
Time: 0.087 sec
Unit 1643
Time: 0.088 sec
Unit 1644
Time: 0.063 sec
Unit 1645
Time: 0.061 sec
Unit 1646
Time: 0.061 sec
Unit 1647
Time: 0.060 sec
Unit 1648
Time: 0.062 sec
Unit 1649
Time: 0.066 sec
Unit 1650
Time: 0.062 sec
Unit 1651
Time: 0.059 sec
Unit 1652
Time: 0.063 sec
Unit 1653
Time: 0.170 sec
Unit 1654
Time: 0.081 sec
Unit 1655
Time: 0.063 sec
Unit 1656
Time: 0.063 sec
Unit 1657
Time: 0.061 sec
Unit 1658
Time: 0.058 sec
Unit 1659
Time: 0.059 sec
Unit 1660
Time: 0.061 sec
Unit 1661
Time: 0.061 sec
Unit 1662
Time: 0.062 

Time: 0.057 sec
Unit 1942
Time: 0.056 sec
Unit 1943
Time: 0.057 sec
Unit 1944
Time: 0.055 sec
Unit 1945
Time: 0.063 sec
Unit 1946
Time: 0.058 sec
Unit 1947
Time: 0.055 sec
Unit 1948
Time: 0.060 sec
Unit 1949
Time: 0.064 sec
Unit 1950
Time: 0.156 sec
Unit 1951
Time: 0.102 sec
Unit 1952
Time: 0.077 sec
Unit 1953
Time: 0.060 sec
Unit 1954
Time: 0.061 sec
Unit 1955
Time: 0.063 sec
Unit 1956
Time: 0.058 sec
Unit 1957
Time: 0.058 sec
Unit 1958
Time: 0.063 sec
Unit 1959
Time: 0.061 sec
Unit 1960
Time: 0.062 sec
Unit 1961
Time: 0.061 sec
Unit 1962
Time: 0.063 sec
Unit 1963
Time: 0.069 sec
Unit 1964
Time: 0.098 sec
Unit 1965
Time: 0.082 sec
Unit 1966
Time: 0.060 sec
Unit 1967
Time: 0.060 sec
Unit 1968
Time: 0.059 sec
Unit 1969
Time: 0.064 sec
Unit 1970
Time: 0.058 sec
Unit 1971
Time: 0.057 sec
Unit 1972
Time: 0.057 sec
Unit 1973
Time: 0.066 sec
Unit 1974
Time: 0.115 sec
Unit 1975
Time: 0.156 sec
Unit 1976
Time: 0.066 sec
Unit 1977
Time: 0.065 sec
Unit 1978
Time: 0.056 sec
Unit 1979
Time: 0.064 

In [29]:

print(y_predicted.shape)
print(y_true.shape)


(526, 2048)
(526, 2048)


In [37]:

#delta = y_true[20] - y_predicted[20]
#plt.plot(delta)
print(y_true[20])



[0.23404711 0.20910916 0.1618503  ... 0.49356055 0.3529139  0.05369953]


In [38]:
print(y_predicted[20])


[0.29889751 0.50857443 0.10412158 ... 1.17196663 0.65585454 0.54753516]


In [67]:

# Train a classifier on y_true > cat 
# see how well it does on y_pred > cat ..use y_pred as test

del classifier_model
X_input = Input(shape=(2048))
#xr = Flatten()(x)
#X = BatchNormalization()(X_input)
#xr = Dense(1024, activation='relu')(xr)
#xr = Dropout(0.2)(xr)
#xr = Dense(1024, activation='softmax')(xr)
#xr = GlobalAveragePooling2D()(x)
X = Dense(1024, activation='relu')(X_input)
X = Dropout(0.4)(X)
#X = BatchNormalization()(X)
X = Dense(16, activation='relu')(X)
X = Dropout(0.2)(X)
#X = BatchNormalization()(X)
#predictions = Dense(17, activation='softmax')(xr)
predictions = Dense(3, activation='softmax')(X)
classifier_model = Model(inputs=X_input, outputs=predictions)
classifier_model.summary()


Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 2048)]            0         
_________________________________________________________________
dense_27 (Dense)             (None, 1024)              2098176   
_________________________________________________________________
dropout_14 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_28 (Dense)             (None, 16)                16400     
_________________________________________________________________
dropout_15 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_29 (Dense)             (None, 3)                 51        
Total params: 2,114,627
Trainable params: 2,114,627
Non-trainable params: 0
________________________________________________

In [68]:

#features_train = y_true
#labels_train = labels_all[div:]
features_train = y_train
labels_train = labels_all[:div]

features_test = y_test
labels_test = labels_all[div:]

EPOCHS=50
callbacks = [TensorBoard(log_dir=f'./log/{i}')]
classifier_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
classifier_model.fit(x=features_train, y=labels_train, shuffle=True, epochs=EPOCHS, callbacks=callbacks, validation_data=(features_test, labels_test))



Train on 4728 samples, validate on 526 samples
Epoch 1/50
4728/4728 [==============================] - 3s 625us/sample - loss: 0.9918 - accuracy: 0.4450 - val_loss: 0.8621 - val_accuracy: 0.5019
Epoch 2/50
4728/4728 [==============================] - 2s 492us/sample - loss: 0.8654 - accuracy: 0.5127 - val_loss: 0.8029 - val_accuracy: 0.5608
Epoch 3/50
4728/4728 [==============================] - 2s 435us/sample - loss: 0.8334 - accuracy: 0.5590 - val_loss: 0.7322 - val_accuracy: 0.6502
Epoch 4/50
4728/4728 [==============================] - 2s 433us/sample - loss: 0.8002 - accuracy: 0.6083 - val_loss: 0.7032 - val_accuracy: 0.7186
Epoch 5/50
4728/4728 [==============================] - 2s 437us/sample - loss: 0.7398 - accuracy: 0.6743 - val_loss: 0.7398 - val_accuracy: 0.7091
Epoch 6/50
4728/4728 [==============================] - 2s 434us/sample - loss: 0.7375 - accuracy: 0.6899 - val_loss: 0.8127 - val_accuracy: 0.7510
Epoch 7/50
4728/4728 [==============================] - 2s 430us/